# Demo: Raster fundamentals, Rasterio, Band Math with Arrays
UW Geospatial Data Analysis  
CEE467/CEWA567  
David Shean  

## Introduction
See reading assignment [05_Raster1_GDAL_rasterio_LS8_prep](./05_Raster1_GDAL_rasterio_LS8_prep.md)

## What is a raster?

## Raster data sources
* Satellite imagery
* Gridded model output
* Interpolated vector data

## Raster fundamentals
Interactive discussion during demo

### Dimensions (width [columns] and height [rows] in pixels)
### CRS (coordinate system)
### Extent (bounds)
### Resolution (pixel size)
### Data type (bit depth)
### Number of bands
### NoData values

## GDAL (Geospatial Data Abstraction Library) and Rasterio
GDAL is a powerful and mature library for reading, writing and warping raster datasets, written in C++ with bindings to other languages. There are a variety of geospatial libraries available on the python package index, and almost all of them depend on GDAL. One such python library developed and supported by Mapbox, rasterio, builds on top of GDAL’s many features, but provides a more pythonic interface and supports many of the features and formats that GDAL supports. Both GDAL and rasterio are constantly being updated and improved.

## Raster formats
* GeoTiff is most common
* GDAL is the foundation - drivers for hundreds of formats

## CRS and Projections
* Most often UTM
* PROJ is the foundation (as with GeoPandas)

## Raster transformations

### Sensor to raster image
* Simple example of a 2D CCD/CMOS detector in a simple camera (e.g., Planet Dove)
* Snapshot of the Earth
* Sensor model allows you to relate each pixel in the image to a geographic location on the ground
* Existing DEM to produce an orthorectified image in some projected coordinate system
* That's where we start

### Raster image (lines, samples) to projected coordinates
* Need a way to relate from pixel coordinates (2D rectangular image on your screen) to real-world coordinates (projected)
    * Pixel coordinates: image width, height in units of pixels, starting at (0,0)
    * Real-world coordiantes: projected coordinate system (e.g., UTM 10N), units of meters
* Origin is usually upper left corner of upper left pixel
    * Careful about this - you will definitely run into this problem at some point
    * Often your grid may be shifted by a half a pixel in x and y
* Negative y cell size - what's up with that?

### GDAL/ESRI affine

### rasterio affine
* Multiply affine by raster indices to get projected coordinates
* Rasterio dataset `xy` and `index` methods

## Basic raster structure
* Dataset
* Bands
    * Often just 1 band, sometimes multiple bands (new axis)
* Read band to get underlying 2D array data
    * Handling missing data (nodata) - masked arrays vs. np.nan

### Overviews

## 3D array to create composites from multispectral bands
* Can dstack 2D arrays

## Misc
* Be careful with large rasters, esp float - don't load into memory
* Read in a window or every nth pixel when prototyping. Only read in full res when ready.
* Try to avoid creating many copies of arrays

## GDAL command line utilities

* Learn these: https://gdal.org/programs/index.html
    * gdalinfo
    * gdal_translate
    * gdalwarp
    * gdaladdo
* Items to discuss
    * Use standard creation options (co)
        * TILED=YES
        * COMPRESS=LZW
        * BIGTIFF=IF_SAFER
    * Resampling algorithms
        * Default is nearest
        * Often bilinear or bicubic is a better choice for reprojecting, upsampling, downsampling

https://medium.com/planet-stories/a-gentle-introduction-to-gdal-part-4-working-with-satellite-data-d3835b5e2971

# Demo

https://automating-gis-processes.github.io/site/notebooks/Raster/reading-raster.html

In [ ]:
pwd

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import rasterio as rio
import rasterio.plot
from osgeo import gdal

#Useful package to add dynamic scalebar to matplotlib images
from matplotlib_scalebar.scalebar import ScaleBar

In [ ]:
#We want to use interactive plotting for zoom/pan and live coordinate display
#matplotlib widget
%matplotlib inline

In [ ]:
#Create local directory to store images
imgdir = 'LS8_sample'

#Pre-identified cloud-free Image IDs used for the lab
#Summer 2018
img_id1 = 'LC08_L2SP_046027_20180818_20200831_02_T1'
#Winter 2018
img_id2 = 'LC08_L2SP_046027_20181224_20200829_02_T1'

img = img_id1

## Specify filenames for specific bands used for later examples

In [ ]:
#Specify filenames for different bands we will need for the lab
#Check table from background section to see wavelengths of each band number

#Red
r_fn = os.path.join(imgdir, img+'_SR_B4.TIF')
#Green
g_fn = os.path.join(imgdir, img+'_SR_B3.TIF')
#Blue
b_fn = os.path.join(imgdir, img+'_SR_B2.TIF')
#Near-Infrared
nir_fn = os.path.join(imgdir, img+'_SR_B5.TIF')
#Shortwave-Infrared
swir_fn = os.path.join(imgdir, img+'_SR_B6.TIF')
#Panchromatic
p_fn = os.path.join(imgdir, img+'_SR_B8.TIF')

In [ ]:
#Specify filenames for different bands we will need for the lab
#Check table from background section to see wavelengths of each band number
tir_fn = os.path.join(imgdir, img+'_ST_B10.TIF')
print(tir_fn)

In [ ]:
!gdalinfo $tir_fn

# Rasterio basics
* We'll stick with rasterio for most of our Python raster analysis 
* https://rasterio.readthedocs.io/en/stable/quickstart.html

## Use a Python `with` construct to cleanly open, inspect, and close the file directly from the url
* The Python `with` construct may be new, or maybe you used it during Lab02 when opening a text file for reading/writing.
* It is "used in exception handling to make the code cleaner and much more readable. It simplifies the management of common resources like file streams."
    * Enables more elegant file opening/closing and handling errors (like missing files)
* Let's use the `with rio.open()` approach to print out the rasterio dataset profile, without actually reading the underlying image data
    * We will temporarily store the rasterio dataset with variable name `src` (short for "source")

In [ ]:
with rio.open(tir_fn) as src:
    print(src.profile)

## Can also open dataset with rasterio for persistence and interactive access
* This is likely a better option as you're learning, as you can access the opened dataset and arrays you've already read in other cells
* Remember to close the rasterio dataset when no longer needed!

In [ ]:
src = rio.open(tir_fn)

In [ ]:
#Notes on attemting to apply scale and offset dynamically
#mode='r+', scales=(st_scale,), offsets=(st_offset,), mask_and_scale=True
#src.scales = (st_scale,)
#src.offsets = (st_offset,)
#src.scales, src.offsets

In [ ]:
type(src)

In [ ]:
src.profile

In [ ]:
src.meta

In [ ]:
src.crs

### Plot using rasterio `show()` function
* Note axes tick labels

In [ ]:
rio.plot.show(src);

### Read the array

In [ ]:
#src.read?

In [ ]:
#Note memory usage before and after reading
%time
a = src.read(1)

In [ ]:
a

### Plot using Matplotlib `imshow`
* Note axes tick labels

In [ ]:
f,ax = plt.subplots()
ax.imshow(a);

## Inspect the array

In [ ]:
a.shape

In [ ]:
a.size

In [ ]:
a.dtype

In [ ]:
a.min()

In [ ]:
a.max()

In [ ]:
2**16

In [ ]:
a

In [ ]:
a.ravel()

In [ ]:
f, ax = plt.subplots()
plt.hist(a.ravel(), bins=128);

## Image bit depth
Number of possible intensity values

In [ ]:
#Landsat-8 OLI is 12-bit sensor
2**12

In [ ]:
2**16

## Use a masked array to handle nodata

In [ ]:
src.nodata

In [ ]:
src.nodatavals

In [ ]:
#If nodata is defined, rasterio can created masked array on the fly
a = src.read(1, masked=True)

In [ ]:
a

In [ ]:
np.ma.masked_equal?

In [ ]:
np.ma.masked_equal(a, 0)

In [ ]:
a = np.ma.masked_equal(a, 0)

In [ ]:
f, ax = plt.subplots()
plt.imshow(a);

In [ ]:
f, ax = plt.subplots()
plt.hist(a.ravel(), bins=128);

## Scaling 16-bit values to surface reflectance/temperature
* See conversion factors here: https://www.usgs.gov/landsat-missions/landsat-collection-2-level-2-science-products
* https://www.usgs.gov/faqs/how-do-i-use-scale-factor-landsat-level-2-science-products

### Surface reflectance values from 0.0 to 1.0
### Surface temperature values in Kelvin

In [ ]:
#SR 0.0000275 + -0.2
sr_scale = 0.0000275
sr_offset = -0.2
#ST 0.00341802 + 149.0
st_scale = 0.00341802
st_offset = 149.0

In [ ]:
a_st = a * st_scale + st_offset

In [ ]:
a_st.dtype

In [ ]:
#Float16 should provide enough precision for these data
a_st.astype('float16')

In [ ]:
f, ax = plt.subplots()
plt.hist(a_st.ravel(), bins=128);

In [ ]:
a_st -= 273

In [ ]:
f, ax = plt.subplots()
plt.hist(a_st.compressed(), bins=128);

### Bounds and extent

In [ ]:
#This is rasterio bounds object - note labels like dictionary keys and values
src.bounds

In [ ]:
#This is matplotlib extent
full_extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]
print(full_extent)

In [ ]:
#rasterio convenience function
full_extent = rio.plot.plotting_extent(src)
print(full_extent)

### Plot the image with `imshow`, but now pass in this `extent` as an argument
* Note how the axes coordinates change
    * These should now be meters in the UTM 10N coordinate system of the projected image!

In [ ]:
f, axa = plt.subplots(1,2, figsize=(10,6))
axa[0].imshow(a, cmap='gray') #vmin=0, vmax=1
axa[0].set_title("Array coordinates")
axa[1].imshow(a, extent=full_extent, cmap='gray') #vmin=0, vmax=1
axa[1].set_title("Projected coordinates")
plt.tight_layout()

## Raster transform
* How does rasterio know the bounds of the array?
* Inspect the dataset `transform` attribute
* You may have encountered an ESRI "world file" or GDAL geotransform before. This is the same idea, but Rasterio's model uses traditional affine transform.
* Review this: https://rasterio.readthedocs.io/en/stable/topics/georeferencing.html?highlight=affine#coordinate-transformation

In [ ]:
src.transform

In [ ]:
#These are (x,y) for corners in pixel space
#A bit confusing due to (row,col) of shape, which is (y,x)

#Upper left
ul = (0, 0)
#Lower right
lr = (a_st.shape[1], a_st.shape[0])

In [ ]:
#Transform upper left corner
ul_proj = src.transform * ul
ul_proj

In [ ]:
#Transform lower right corner
lr_proj = src.transform * lr
lr_proj

In [ ]:
wh_km = np.abs(np.array(ul_proj) - np.array(lr_proj))/1000
wh_km
print('Total width: %0.2f km\nTotal height: %0.2f km' % (wh_km[0], wh_km[1]))

## Raster and array sampling
* Use helper functions `xy` and `sample`

In [ ]:
#Array coordinates
c = (3512, 3512)

In [ ]:
a_st[3000,3000]

In [ ]:
#Get value at coordinates using array indexing
a_st[c[0], c[1]]

In [ ]:
#src.xy?

In [ ]:
#Note use of argument expansion here (*c) so we don't have to pass individual c[0] and c[1] values
x,y = src.xy(*c)
print(x,y)

In [ ]:
#src.sample?

In [ ]:
src.sample(x, y)

In [ ]:
#Doesn't work
#list(src.sample(x, y))

In [ ]:
src.sample([(x, y),])

In [ ]:
#Pass in a list, and evaluate the generator
list(src.sample([(x, y),]))

## Windowing and indexing

In [ ]:
chunk = a_st[3000:4024,3000:4024]
chunk

In [ ]:
f, ax = plt.subplots()
plt.imshow(chunk)

### Store a reduced resolution view (1 pixel for every 100 original pixels)

In [ ]:
asub = a_st[::10, ::10]
asub

In [ ]:
asub.shape

In [ ]:
#Every 10th pixel - great strategy for quick visualization during development/exploration
f, ax = plt.subplots()
plt.imshow(asub);

## Raster math

In [ ]:
#%matplotlib widget

In [ ]:
#Remember to use compressed for historgrams with 2D masked arrays
f, ax = plt.subplots()
plt.hist(asub.compressed(), bins=128);

In [ ]:
t_thresh = 18

In [ ]:
asub < t_thresh

In [ ]:
f, ax = plt.subplots()
plt.imshow(asub <= t_thresh);

In [ ]:
plt.imshow(asub <= t_thresh, interpolation='none');

## Calculating area

In [ ]:
(asub <= t_thresh).sum()

In [ ]:
src.res

## Cleanup and memory management

In [ ]:
#Delete array from memory
asub = None
a = None
a_st = None
#Close the rasterio dataset
src.close()

## GDAL Python API basics
* I'm including this for reference
    * It's not that complicated, even though rasterio is the more popular option for Python these days (partly because of much better documentation)
* https://gdal.org/user/raster_data_model.html
* https://github.com/OSGeo/gdal/tree/master/gdal/swig/python/samples
* https://pcjericks.github.io/py-gdalogr-cookbook/index.html

In [ ]:
#Open the green band GeoTiff as GDAL Dataset object
ds = gdal.Open(tir_fn)

In [ ]:
#Get the raster band
gdal_b = ds.GetRasterBand(1)
#Read into array
a = gdal_b.ReadAsArray()

In [ ]:
#Inspect the array
a

In [ ]:
#View the array
f, ax = plt.subplots()
ax.imshow(a);

In [ ]:
#Set array to None (frees up RAM) and close GDAL dataset
a = None
ds = None